# Diffbot Graph Transformer | Diffbot Graph Transformer[![Colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/graph/diffbot_graphtransformer.ipynb)> [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/graph/diffbot_graphtransformer.ipynb)## Use case | ユースケーステキストデータには、様々な分析、推薦システム、または知識管理アプリケーションに有用な豊かな関係性や洞察がしばしば含まれています。> Text data often contain rich relationships and insights that can be useful for various analytics, recommendation engines, or knowledge management applications.DiffbotのNLP APIは、非構造化テキストデータからエンティティ、関係性、およびセマンティックな意味を抽出することが可能です。> Diffbot's NLP API allows for the extraction of entities, relationships, and semantic meaning from unstructured text data.DiffbotのNLP APIとNeo4j（グラフデータベース）を組み合わせることで、テキストから抽出された情報に基づいて強力で動的なグラフ構造を作成することができます。これらのグラフ構造は完全にクエリ可能であり、様々なアプリケーションに統合可能です。> By coupling Diffbot's NLP API with Neo4j, a graph database, you can create powerful, dynamic graph structures based on the information extracted from text. These graph structures are fully queryable and can be integrated into various applications.この組み合わせにより、以下のような使用例が可能になります：> This combination allows for use cases such as:* テキスト文書、ウェブサイト、またはソーシャルメディアのフィードから知識グラフを構築する。  > Building knowledge graphs from textual documents, websites, or social media feeds.* データ内の意味的な関係に基づいて推薦を生成します。  > Generating recommendations based on semantic relationships in the data.* エンティティ間の関係を理解する高度な検索機能を作成する。  > Creating advanced search features that understand the relationships between entities.* ユーザーがデータ内の隠れた関係性を探索できる分析ダッシュボードを構築する。  > Building analytics dashboards that allow users to explore the hidden relationships in data.## Overview | 概要LangChainはグラフデータベースと対話するためのツールを提供しています。> LangChain provides tools to interact with Graph Databases:1. `テキストから知識グラフを構築する`際に、グラフトランスフォーマーとストレージの統合を活用する   > `Construct knowledge graphs from text` using graph transformer and store integrations2. `クエリの作成と実行にチェーンを使用して`グラフデータベースを問い合わせる   > `Query a graph database` using chains for query creation and execution3. `グラフデータベース`を使用して、堅牢で柔軟なクエリングのためのエージェントと対話する   > `Interact with a graph database` using agents for robust and flexible querying## Quickstart | クイックスタートまず、必要なパッケージを取得し、環境変数を設定してください：> First, get required packages and set environment variables:

In [ ]:
!pip install langchain langchain-experimental openai neo4j wikipedia

## Diffbot NLP Service | Diffbot NLP ServiceDiffbotのNLPサービスは、非構造化テキストデータからエンティティ、関係性、および意味的文脈を抽出するツールです。この抽出された情報は、ナレッジグラフを構築するために使用できます。このサービスを利用するには、[Diffbot](https://www.diffbot.com/products/natural-language/)からAPIキーを取得する必要があります。> Diffbot's NLP service is a tool for extracting entities, relationships, and semantic context from unstructured text data.> This extracted information can be used to construct a knowledge graph.> To use their service, you'll need to obtain an API key from [Diffbot](https://www.diffbot.com/products/natural-language/).

In [2]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer

diffbot_api_key = "DIFFBOT_API_KEY"
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

このコードは「Warren Buffett」に関するWikipediaの記事を取得し、その後`DiffbotGraphTransformer`を使用してエンティティと関係性を抽出します。`DiffbotGraphTransformer`は構造化されたデータ`GraphDocument`を出力し、これをグラフデータベースに反映させることができます。Diffbotの[APIリクエストごとの文字数制限](https://docs.diffbot.com/reference/introduction-to-natural-language-api)により、テキストチャンキングは避けられています。> This code fetches Wikipedia articles about "Warren Buffett" and then uses `DiffbotGraphTransformer` to extract entities and relationships.> The `DiffbotGraphTransformer` outputs a structured data `GraphDocument`, which can be used to populate a graph database.> Note that text chunking is avoided due to Diffbot's [character limit per API request](https://docs.diffbot.com/reference/introduction-to-natural-language-api).

In [3]:
from langchain.document_loaders import WikipediaLoader

query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

## Loading the data into a knowledge graph | ナレッジグラフへのデータの読み込みNeo4jのインスタンスを実行中である必要があります。一つの選択肢として、[Auraクラウドサービスで無料のNeo4jデータベースインスタンスを作成する](https://neo4j.com/cloud/platform/aura-graph-database/)ことができます。また、[Neo4jデスクトップアプリケーション](https://neo4j.com/download/)を使用してローカルでデータベースを実行するか、Dockerコンテナを使用することもできます。ローカルでDockerコンテナを実行するには、次のスクリプトを実行します：> You will need to have a running Neo4j instance. One option is to create a [free Neo4j database instance in their Aura cloud service](https://neo4j.com/cloud/platform/aura-graph-database/). You can also run the database locally using the [Neo4j Desktop application](https://neo4j.com/download/), or running a docker container. You can run a local docker container by running the executing the following script:```docker run \    --name neo4j \    -p 7474:7474 -p 7687:7687 \    -d \    -e NEO4J_AUTH=neo4j/pleaseletmein \    -e NEO4J_PLUGINS=\[\"apoc\"\]  \    neo4j:latest```Dockerコンテナを使用している場合は、データベースが起動するまで数秒間待つ必要があります。> If you are using the docker container, you need to wait a couple of second for the database to start.

In [4]:
from langchain.graphs import Neo4jGraph

url = "bolt://localhost:7687"
username = "neo4j"
password = "pleaseletmein"

graph = Neo4jGraph(url=url, username=username, password=password)

`GraphDocuments`は、`add_graph_documents`メソッドを使用して知識グラフにロードできます。> The `GraphDocuments` can be loaded into a knowledge graph using the `add_graph_documents` method.

In [5]:
graph.add_graph_documents(graph_documents)

## Refresh graph schema information | グラフスキーマ情報を更新するデータベースのスキーマが変更された場合、Cypherステートメントを生成するために必要なスキーマ情報を更新できます> If the schema of database changes, you can refresh the schema information needed to generate Cypher statements

In [6]:
graph.refresh_schema()

## Querying the graph | グラフの問い合わせグラフサイファーQAチェーンを使用してグラフに質問をすることができるようになりました。最高の体験を得るためには、Cypherクエリを構築する際に**GPT-4**を使用することをお勧めします。> We can now use the graph cypher QA chain to ask question of the graph. It is advisable to use **gpt-4** to construct Cypher queries to get the best experience.

In [7]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    graph=graph,
    verbose=True,
)

In [8]:
chain.run("Which university did Warren Buffett attend?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Warren Buffett"})-[:EDUCATED_AT]->(o:Organization)
RETURN o.name
Full Context:
[{'o.name': 'New York Institute of Finance'}, {'o.name': 'Alice Deal Junior High School'}, {'o.name': 'Woodrow Wilson High School'}, {'o.name': 'University of Nebraska'}]

> Finished chain.


'Warren Buffett attended the University of Nebraska.'

In [9]:
chain.run("Who is or was working at Berkshire Hathaway?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[r:EMPLOYEE_OR_MEMBER_OF]->(o:Organization) WHERE o.name = 'Berkshire Hathaway' RETURN p.name
Full Context:
[{'p.name': 'Charlie Munger'}, {'p.name': 'Oliver Chace'}, {'p.name': 'Howard Buffett'}, {'p.name': 'Howard'}, {'p.name': 'Susan Buffett'}, {'p.name': 'Warren Buffett'}]

> Finished chain.


'Charlie Munger, Oliver Chace, Howard Buffett, Susan Buffett, and Warren Buffett are or were working at Berkshire Hathaway.'